In [1]:
import pandas as pd 
import os
import yfinance as yf
import spacy
from heapq import nlargest
import bs4 as bs
import requests
from spacy import displacy
from collections import Counter
import matplotlib
from string import punctuation
import json

In [2]:
data = pd.read_csv('./us_equities_news_dataset.csv')

In [3]:
with open('./sp500infos.json') as json_file:
    sp500 = json.load(json_file)

In [4]:
data.columns

Index(['id', 'ticker', 'title', 'category', 'content', 'release_date',
       'provider', 'url', 'article_id'],
      dtype='object')

In [5]:
data.iloc[0]

id                                                         221515
ticker                                                        NIO
title           Why Shares of Chinese Electric Car Maker NIO A...
category                                                     news
content         What s happening\nShares of Chinese electric c...
release_date                                           2020-01-15
provider                                          The Motley Fool
url                                        https://invst.ly/pigqi
article_id                                                2060327
Name: 0, dtype: object

In [6]:
data['category'].unique()

array(['news', 'opinion'], dtype=object)

In [7]:
resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})
tickers = []
for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    tickers.append(ticker)
tickers = [s.replace('\n', '') for s in tickers]
print(len(tickers))

505


In [8]:
fast_lookup = dict.fromkeys(tickers, 1)
data.set_index('ticker')
to_remove = []
for i in range(len(data)):
    if (i%10000==0):
        print(i)
    if data.iloc[i]['ticker'] not in fast_lookup:
        to_remove.append(i)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000


In [9]:
data = data.drop(to_remove)

In [10]:
len(data['ticker'].unique())


349

In [11]:
len(data)

179279

In [12]:
data

,id,ticker,title,category,content,release_date,provider,url,article_id
1302,222817,INTC,Intel INTC To Post Q4 Earnings DCG CCG In...,opinion,Intel NASDAQ INTC is scheduled to report f...,2020-01-20,Zacks Investment Research,https://www.investing.com/analysis/intel-intc-...,200500288
1303,222818,INTC,Stocks Jump About 2 This Week On Trade Deals ...,opinion,The major indices just finished another positi...,2020-01-20,Zacks Investment Research,https://www.investing.com/analysis/stocks-jump...,200500165
1304,222819,INTC,Apple Partner Pegatron Plans New Facilities In...,opinion,Apple s NASDAQ AAPL assembly partner Pega...,2020-01-20,Zacks Investment Research,https://www.investing.com/analysis/apple-partn...,200500270
1305,222820,MSFT,Microsoft exposed 250M customer support records,news,Microsoft MSFT 0 1 discloses a database er...,2020-01-23,Seeking Alpha,https://invst.ly/pli4a,2065317
1306,222821,MU,Wall Street dips as investors focus on Middle ...,news,By Caroline Valetkevitch NEW YORK Reuters ...,2020-01-07,Reuters,https://www.investing.com/news/stock-market-ne...,2053896
...,...,...,...,...,...,...,...,...,...
221508,443024,T,Crude And Steel Still In Sync,opinion,We have been reporting on the trade off betwee...,2012-10-04,Ivan Kitov,https://www.investing.com/analysis/crude-and-s...,138733
221509,443025,T,Forget AT T This Is The Telecom Stock You Sho...,opinion,It s the largest cell phone provider in the wo...,2012-05-30,StreetAuthority,https://www.investing.com/analysis/forget-at-t...,124829
221510,443026,T,Wall Street Exposed Part 3 How Dividends C...,opinion,Before we dicuss how the mechanism of dividend...,2012-07-16,Portfolio Cafe,https://www.investing.com/analysis/wall-street...,129651
221511,443027,T,Weighing The Week Ahead It s All About Jobs,opinion,From start to finish the coming week will hav...,2012-09-02,Jeff Miller,https://www.investing.com/analysis/weighing-th...,134926


In [13]:
data.to_csv('us_equities_news_dataset_clean.csv')